# 👤 YOLO11n Face Detection & Recognition Training

**Petrol Pump Deployment - Employee & Customer Face Analytics**

**Multiple Public Datasets: Roboflow + Kaggle + WIDER FACE**

**Run cells in order!**

## 📋 Use Cases for Petrol Pump

| Use Case | Description |
|----------|-------------|
| **Employee Attendance** | Auto clock-in/out via face recognition |
| **Unknown Person Alert** | Detect unfamiliar faces in restricted areas |
| **Customer Analytics** | Count unique customers, demographics |
| **VIP Detection** | Alert when regular/VIP customers arrive |
| **Theft Prevention** | Flag known offenders |
| **Safety Compliance** | Verify employee identity at pumps |

## 1. Setup & GPU Check

In [ ]:
# Install packages
!pip install -q ultralytics>=8.3.0 roboflow gdown opencv-python-headless kaggle

# Check GPU
!nvidia-smi

import torch
print(f"\n✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Core Imports
import os
import glob
import shutil
import yaml
import torch
import zipfile
import random
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from IPython.display import display, Image as IPImage, clear_output

# Paths
MERGED_PATH = "/content/face_detection_dataset"
RESULTS_DIR = "/content/face_detection_results"

print("✅ All imports ready!")

## 2. Download Public Face Datasets

We'll download from multiple verified public sources:
1. **Roboflow WIDER Face** - Large-scale benchmark (WORKS ✅)
2. **Kaggle Face Detection** - 16.7k images with YOLO annotations
3. **FDDB Roboflow** - Classic benchmark converted to YOLO
4. **YOLO Face Detection** - Verified Roboflow dataset
5. **WIDER Face Original** - HuggingFace (WORKS ✅)

In [ ]:
from roboflow import Roboflow

# ========================================
# 🔑 PASTE YOUR ROBOFLOW API KEY HERE
# Get it from: https://app.roboflow.com/settings/api
# ========================================
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print("✅ Connected to Roboflow!")

In [ ]:
# ========================================
# 📊 DATASET 1: WIDER Face Detection (Roboflow) - VERIFIED WORKING
# Large-scale dataset with 32k+ images
# ========================================
print("📥 Downloading Dataset 1: WIDER Face (Roboflow)...")

try:
    project1 = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
    dataset1 = project1.version(18).download("yolov8")
    print(f"✅ Dataset 1 (WIDER Face): {dataset1.location}")
except Exception as e:
    print(f"⚠️ Dataset 1 failed: {e}")
    dataset1 = None

In [ ]:
# ========================================
# 📊 DATASET 2: Kaggle Face Detection (16.7k images) - NEW
# Direct download with YOLO annotations
# Source: https://kaggle.com/datasets/fareselmenshawii/face-detection-dataset
# ========================================
print("📥 Downloading Dataset 2: Kaggle Face Detection...")
print("\n⚠️ Kaggle requires authentication!")
print("   1. Go to https://www.kaggle.com/settings")
print("   2. Click 'Create New Token' to download kaggle.json")
print("   3. Upload kaggle.json when prompted below\n")

from google.colab import files

# Check if kaggle.json already exists
os.makedirs('/root/.kaggle', exist_ok=True)
kaggle_json_path = '/root/.kaggle/kaggle.json'

try:
    if not os.path.exists(kaggle_json_path):
        print("📤 Please upload your kaggle.json file:")
        uploaded = files.upload()
        
        for filename in uploaded.keys():
            if filename == 'kaggle.json':
                with open(kaggle_json_path, 'wb') as f:
                    f.write(uploaded[filename])
                os.chmod(kaggle_json_path, 0o600)
                print("✅ Kaggle credentials saved!")
    
    # Download dataset
    !kaggle datasets download -d fareselmenshawii/face-detection-dataset -p /content/dataset2_kaggle --unzip -q
    
    dataset2_path = '/content/dataset2_kaggle'
    if os.path.exists(dataset2_path) and len(os.listdir(dataset2_path)) > 0:
        print(f"✅ Dataset 2 (Kaggle): {dataset2_path}")
        !ls {dataset2_path}
    else:
        dataset2_path = None
        print("⚠️ Dataset 2 directory empty")
        
except Exception as e:
    print(f"⚠️ Dataset 2 (Kaggle) failed: {e}")
    print("   You can skip this and continue with other datasets")
    dataset2_path = None

In [ ]:
# ========================================
# 📊 DATASET 3: FDDB-style Face Detection (Roboflow) - NEW
# Various public face detection datasets
# ========================================
print("📥 Downloading Dataset 3: Roboflow Face Datasets...")

dataset3 = None

# Try multiple Roboflow sources
roboflow_sources = [
    ("human-faces-object-detection", "faces-t8ou2", 1, "Human Faces"),
    ("selfdriving-car-4ithn", "face-detection-t4v2d", 2, "Car Face Detection"),
    ("facial-emotion", "yolo-face-detection", 1, "YOLO Face"),
    ("yolo-project-0jqzr", "face-detection-urcfj", 1, "YOLO Project"),
]

for workspace, project_name, version, desc in roboflow_sources:
    try:
        print(f"   Trying: {desc}...")
        proj = rf.workspace(workspace).project(project_name)
        dataset3 = proj.version(version).download("yolov8")
        print(f"✅ Dataset 3 ({desc}): {dataset3.location}")
        break
    except Exception as e:
        print(f"   ⚠️ {desc} failed: {str(e)[:50]}...")
        continue

if dataset3 is None:
    print("⚠️ All Dataset 3 sources failed - continuing without it")

In [ ]:
# ========================================
# 📊 DATASET 4: Additional Face Detection (Roboflow) - NEW
# More face detection datasets
# ========================================
print("📥 Downloading Dataset 4: Additional Face Detection...")

dataset4 = None

# Try additional sources
additional_sources = [
    ("face-detection-yolo8n", "face-detection-u2umf", 1, "Face Detection YOLOv8n"),
    ("face-detection-mnspc", "face-detection-wpjtl", 1, "Face Detection 2"),
    ("face-detection-ehwbh", "face-detection-kpsbl", 1, "Face Detection 3"),
]

for workspace, project_name, version, desc in additional_sources:
    try:
        print(f"   Trying: {desc}...")
        proj = rf.workspace(workspace).project(project_name)
        dataset4 = proj.version(version).download("yolov8")
        print(f"✅ Dataset 4 ({desc}): {dataset4.location}")
        break
    except Exception as e:
        print(f"   ⚠️ {desc} failed: {str(e)[:50]}...")
        continue

if dataset4 is None:
    print("⚠️ All Dataset 4 sources failed - continuing without it")

In [ ]:
# ========================================
# 📊 DATASET 5: WIDER Face from HuggingFace - VERIFIED WORKING
# Download WIDER Face benchmark dataset
# ========================================
print("📥 Downloading Dataset 5: WIDER Face (HuggingFace)...")

try:
    # Download WIDER Face training set
    !wget -q --show-progress https://huggingface.co/datasets/wider_face/resolve/main/data/WIDER_train.zip -O /content/wider_train.zip
    
    # Extract
    with zipfile.ZipFile('/content/wider_train.zip', 'r') as z:
        z.extractall('/content/dataset5')
    
    dataset5_path = '/content/dataset5'
    print(f"✅ Dataset 5 (WIDER Original): {dataset5_path}")
    !ls {dataset5_path}
except Exception as e:
    print(f"⚠️ Dataset 5 failed: {e}")
    dataset5_path = None

In [ ]:
# Check what we have downloaded
print("\n" + "="*60)
print("📊 DOWNLOAD SUMMARY")
print("="*60)

total_downloaded = 0

# Check Roboflow datasets
for i, (ds, name) in enumerate([
    (dataset1, "WIDER Face (Roboflow)"),
    (dataset3, "Roboflow Face Detection"),
    (dataset4, "Additional Face Detection")
], 1):
    if ds is not None:
        try:
            train_imgs = len(glob.glob(f"{ds.location}/train/images/*"))
            valid_imgs = len(glob.glob(f"{ds.location}/valid/images/*"))
            total = train_imgs + valid_imgs
            print(f"Dataset {i} ({name}): ✅ {total} images")
            total_downloaded += total
        except:
            print(f"Dataset {i} ({name}): ⚠️ Could not count")
    else:
        print(f"Dataset {i} ({name}): ❌ Not downloaded")

# Check Kaggle dataset
if dataset2_path and os.path.exists(dataset2_path):
    kaggle_imgs = len(glob.glob(f"{dataset2_path}/**/*.jpg", recursive=True))
    kaggle_imgs += len(glob.glob(f"{dataset2_path}/**/*.png", recursive=True))
    print(f"Dataset 2 (Kaggle): ✅ {kaggle_imgs} images")
    total_downloaded += kaggle_imgs
else:
    print(f"Dataset 2 (Kaggle): ❌ Not downloaded")

# Check WIDER Face HuggingFace
if dataset5_path and os.path.exists(dataset5_path):
    print(f"Dataset 5 (WIDER HuggingFace): ✅ Downloaded")
else:
    print(f"Dataset 5 (WIDER HuggingFace): ❌ Not downloaded")

print(f"\n📊 Total images downloaded: ~{total_downloaded:,}+")

## 3. Prepare & Merge Datasets

Unify all datasets into a single YOLO-compatible format with consistent class IDs.

In [ ]:
# Create merged directory structure
for split in ['train', 'valid', 'test']:
    os.makedirs(f"{MERGED_PATH}/images/{split}", exist_ok=True)
    os.makedirs(f"{MERGED_PATH}/labels/{split}", exist_ok=True)

# Unified class mapping - all face variants map to class 0
CLASS_MAP = {
    'face': 0, 'Face': 0, 'FACE': 0,
    'human_face': 0, 'human-face': 0,
    'person_face': 0, 'head': 0,
    0: 0, '0': 0,
}

# Single class output
CLASSES = {0: 'face'}
print(f"✅ Unified Classes: {CLASSES}")

In [ ]:
def merge_roboflow_dataset(ds, prefix):
    """Merge a Roboflow-downloaded dataset into the unified format."""
    if ds is None:
        return 0
    
    src = ds.location
    count = 0
    
    yaml_path = f"{src}/data.yaml"
    if not os.path.exists(yaml_path):
        print(f"   ⚠️ No data.yaml found at {yaml_path}")
        return 0
    
    for split in ['train', 'valid', 'test']:
        img_dir = f"{src}/{split}/images"
        lbl_dir = f"{src}/{split}/labels"
        
        if not os.path.exists(img_dir):
            continue
        
        for img_path in glob.glob(f"{img_dir}/*"):
            name = os.path.basename(img_path)
            base, ext = os.path.splitext(name)
            
            if ext.lower() not in ['.jpg', '.jpeg', '.png', '.bmp', '.webp']:
                continue
            
            new_name = f"{prefix}_{base}{ext}"
            shutil.copy(img_path, f"{MERGED_PATH}/images/{split}/{new_name}")
            
            lbl_path = f"{lbl_dir}/{base}.txt"
            if os.path.exists(lbl_path):
                with open(lbl_path, 'r') as f:
                    lines = f.readlines()
                
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # Map all classes to 0 (face)
                        new_lines.append(f"0 {' '.join(parts[1:])}\n")
                
                if new_lines:
                    with open(f"{MERGED_PATH}/labels/{split}/{prefix}_{base}.txt", 'w') as f:
                        f.writelines(new_lines)
            
            count += 1
    
    return count

def merge_kaggle_dataset(dataset_path, prefix):
    """Merge Kaggle face detection dataset."""
    if dataset_path is None or not os.path.exists(dataset_path):
        return 0
    
    count = 0
    
    # Find images and labels directories
    for split in ['train', 'val', 'valid', 'test']:
        target_split = 'valid' if split == 'val' else split
        
        # Try different directory structures
        possible_img_dirs = [
            f"{dataset_path}/{split}/images",
            f"{dataset_path}/images/{split}",
            f"{dataset_path}/{split}",
        ]
        
        for img_dir in possible_img_dirs:
            if os.path.exists(img_dir):
                lbl_dir = img_dir.replace('images', 'labels')
                
                for img_path in glob.glob(f"{img_dir}/*"):
                    name = os.path.basename(img_path)
                    base, ext = os.path.splitext(name)
                    
                    if ext.lower() not in ['.jpg', '.jpeg', '.png']:
                        continue
                    
                    new_name = f"{prefix}_{base}{ext}"
                    
                    # Default to train if no valid split structure
                    dest_split = target_split if split != dataset_path else 'train'
                    
                    shutil.copy(img_path, f"{MERGED_PATH}/images/{dest_split}/{new_name}")
                    
                    lbl_path = f"{lbl_dir}/{base}.txt"
                    if os.path.exists(lbl_path):
                        with open(lbl_path, 'r') as f:
                            lines = f.readlines()
                        
                        new_lines = []
                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) >= 5:
                                new_lines.append(f"0 {' '.join(parts[1:])}\n")
                        
                        if new_lines:
                            with open(f"{MERGED_PATH}/labels/{dest_split}/{prefix}_{base}.txt", 'w') as f:
                                f.writelines(new_lines)
                    
                    count += 1
                break
    
    return count

print("✅ Merge functions defined")

In [ ]:
# Merge all datasets
print("🔄 Merging datasets...")
print("="*60)

total_images = 0

# Dataset 1 - WIDER Face (Roboflow)
print("\n📦 Processing Dataset 1 (WIDER Face)...")
n1 = merge_roboflow_dataset(dataset1, "wider")
print(f"   ✅ Merged {n1} images")
total_images += n1

# Dataset 2 - Kaggle
if dataset2_path:
    print("\n📦 Processing Dataset 2 (Kaggle)...")
    n2 = merge_kaggle_dataset(dataset2_path, "kaggle")
    print(f"   ✅ Merged {n2} images")
    total_images += n2

# Dataset 3 - Additional Roboflow
print("\n📦 Processing Dataset 3 (Roboflow)...")
n3 = merge_roboflow_dataset(dataset3, "rf3")
print(f"   ✅ Merged {n3} images")
total_images += n3

# Dataset 4 - Additional Roboflow
print("\n📦 Processing Dataset 4 (Roboflow)...")
n4 = merge_roboflow_dataset(dataset4, "rf4")
print(f"   ✅ Merged {n4} images")
total_images += n4

print("\n" + "="*60)
print(f"✅ TOTAL IMAGES MERGED: {total_images}")

In [ ]:
# Create unified data.yaml
data_yaml_path = f"{MERGED_PATH}/data.yaml"

yaml_content = {
    'path': MERGED_PATH,
    'train': 'images/train',
    'val': 'images/valid',
    'test': 'images/test',
    'nc': 1,
    'names': {0: 'face'}
}

with open(data_yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

# Final stats
train_count = len(glob.glob(f"{MERGED_PATH}/images/train/*"))
valid_count = len(glob.glob(f"{MERGED_PATH}/images/valid/*"))
test_count = len(glob.glob(f"{MERGED_PATH}/images/test/*"))

print(f"✅ Created: {data_yaml_path}")
print(f"\n📊 FINAL DATASET STATISTICS:")
print(f"   Training:   {train_count:,} images")
print(f"   Validation: {valid_count:,} images")
print(f"   Testing:    {test_count:,} images")
print(f"   ─────────────────────")
print(f"   TOTAL:      {train_count + valid_count + test_count:,} images")

## 4. Train YOLO11n Face Detection Model

In [ ]:
# Configuration
MODEL = 'yolo11n.pt'
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 640
PATIENCE = 30

PROJECT_NAME = 'face_detection'
RUN_NAME = 'yolo11n_petrol_pump'

print("="*60)
print("🔧 TRAINING CONFIGURATION")
print("="*60)
print(f"Model:      {MODEL}")
print(f"Epochs:     {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Image Size: {IMG_SIZE}")

In [ ]:
# Load and train
model = YOLO(MODEL)
print(f"✅ Loaded base model: {MODEL}")

print("\n🚀 STARTING TRAINING...")
results = model.train(
    data=data_yaml_path,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    patience=PATIENCE,
    project=PROJECT_NAME,
    name=RUN_NAME,
    exist_ok=True,
    device=0,
    cache=True,
    amp=True,
    plots=True,
)

print("\n✅ TRAINING COMPLETE!")

## 5. Evaluate & Export

In [ ]:
RESULTS_PATH = f"{PROJECT_NAME}/{RUN_NAME}"

# Show results
if os.path.exists(f"{RESULTS_PATH}/results.png"):
    display(IPImage(filename=f"{RESULTS_PATH}/results.png", width=900))

# Validate
best_model = YOLO(f"{RESULTS_PATH}/weights/best.pt")
val = best_model.val(data=data_yaml_path)

print("\n📊 VALIDATION RESULTS")
print(f"mAP50:     {val.box.map50:.4f}")
print(f"mAP50-95:  {val.box.map:.4f}")

In [ ]:
# Export to ONNX
onnx_path = best_model.export(format='onnx', imgsz=IMG_SIZE, simplify=True)
print(f"✅ ONNX exported: {onnx_path}")

In [ ]:
# Download models
from google.colab import files

print("📥 Downloading best.pt...")
files.download(f"{RESULTS_PATH}/weights/best.pt")

In [ ]:
# Download ONNX
print("📥 Downloading ONNX model...")
files.download(onnx_path)

---
## 🎉 Done!

### Deploy on Jetson:
```bash
yolo export model=best.pt format=engine device=0 half=True
```